In [1]:
# Importing Necessary Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_reviews = pd.read_csv("../../data/movie-review-data/rotten_tomatoes_movie_reviews.csv")

In [3]:
df_reviews.sample(1).T

,1386942
id,the_report_2019
reviewId,2650553
creationDate,2019-12-02
criticName,Anne Brodie
isTopCritic,False
originalScore,NaN
reviewState,fresh
publicatioName,What She Said
reviewText,"Driver's dangerous deep dive takes time, patie..."
scoreSentiment,POSITIVE


In [4]:
df_reviews.drop(columns=['reviewId','creationDate', 'isTopCritic', 'reviewState', 'publicatioName', 'reviewUrl'], inplace=True)
df_reviews.sample(2).T

,838276,443531
id,dragged_across_concrete,baby_mama
criticName,Jordan Ruimy,Tricia Olszewski
originalScore,A-,NaN
reviewText,Zahler isn't in the same league yet as Taranti...,After a parade of romantic comedies that shoot...
scoreSentiment,POSITIVE,POSITIVE


In [5]:
df_reviews.shape

(1444963, 5)

In [6]:
df_reviews = df_reviews.drop_duplicates()
df_reviews = df_reviews.reset_index(drop=True)

In [7]:
df_reviews.shape

(1432282, 5)

In [8]:
df_reviews['criticName'].nunique()

15510

In [9]:
df_reviews = df_reviews.groupby('id').agg({
    'originalScore': lambda x: list(x),
    'reviewText': lambda x: dict(zip(
        df_reviews.loc[x.index, 'criticName'],
        x
    )),
    'scoreSentiment': lambda x: {
        'PositiveCount': (x == 'POSITIVE').sum(),
        'NegativeCount': (x == 'NEGATIVE').sum()
    }
}).reset_index()

In [10]:
df_reviews[['PositiveCount', 'NegativeCount']] = pd.DataFrame(
    df_reviews['scoreSentiment'].tolist(), index=df_reviews.index
)

In [11]:
df_reviews['TotalReviews'] = df_reviews['PositiveCount'] + df_reviews['NegativeCount']
df_reviews.drop(columns='scoreSentiment', inplace=True)
df_reviews.head()

,id,originalScore,reviewText,PositiveCount,NegativeCount,TotalReviews
0,$5_a_day,"[2.5/5, D+, 4/5, 2.5/4]","{'Kevin Carr': '$5 a Day isn't perfect, but it...",2,2,4
1,009_re_cyborg,"[2/5, nan, nan, 2/5, 3/5, 2/5, 2/5, 2/5, 4/5, ...","{'Joseph Walsh': 'Despite its good looks, ther...",3,10,13
2,00_mhz,"[5/10, 8.5, 2/5]",{'Caitlin Kennedy': '...a stylishly minimalist...,2,1,3
3,0814255,"[nan, 4/10, 2.0/4.0, 2.5/5, nan, 3/4, 2/4, 3.5...",{'Rene Jordan': 'The Lightning Thief is good j...,73,75,148
4,0878835,"[nan, nan, 3.5/4.0, nan, nan, nan, 8.5/10, nan...",{'Rene Jordan': 'Formless and complicated. [F...,123,19,142


In [12]:
df_reviews = df_reviews[df_reviews['TotalReviews'] >= 10]
df_reviews.head()

,id,originalScore,reviewText,PositiveCount,NegativeCount,TotalReviews
1,009_re_cyborg,"[2/5, nan, nan, 2/5, 3/5, 2/5, 2/5, 2/5, 4/5, ...","{'Joseph Walsh': 'Despite its good looks, ther...",3,10,13
3,0814255,"[nan, 4/10, 2.0/4.0, 2.5/5, nan, 3/4, 2/4, 3.5...",{'Rene Jordan': 'The Lightning Thief is good j...,73,75,148
4,0878835,"[nan, nan, 3.5/4.0, nan, nan, nan, 8.5/10, nan...",{'Rene Jordan': 'Formless and complicated. [F...,123,19,142
6,1-day,"[2/5, 2/5, 4/5, 3/5, 3/5, 2/5, 5/5, 3/5, 4/5, ...","{'Anna Smith': 'A less gripping, if bleaker, t...",8,8,16
8,10,"[4/5, 6/10, 8/10, 2.5/4, 6/10, 4/5, B-, 5/5, n...",{'Eddie Harrison': ''It's probably the writer/...,16,8,24


In [13]:
df_reviews.to_csv('../../data/movie-review-data/transformed_reviews.csv', index=False)